In [1]:
import pandas as pd
import numpy as np
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from dateutil.parser import parse
import time
import os
import pickle

In [2]:
# Anzahl der angezeigten Zeilen in JNP einstellen
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [3]:
# Pickl-Dateipfade vorbereiten:
data_path_weather = '/home/paul/python_projects/masterthesis/data/wetter/'

pkl_file_wetter_step_1 = 'daten_wetter_step_1.p'

In [4]:
df_wetter_pkl = pickle.load(open(data_path_weather + pkl_file_wetter_step_1, 'rb'))

In [5]:
df_wetter_pkl.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,SOURCE,LATITUDE,LONGITUDE,ELEVATION,NAME,REPORT_TYPE,QUALITY_CONTROL,WND,CIG,VIS,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"200,1,N,0036,1","01350,1,C,N","011265,1,N,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"210,1,N,0051,1","01200,1,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0051,1","01350,1,C,N","011265,1,N,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"230,1,N,0051,1","99999,9,C,N","011265,1,N,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,4,52.559686,13.287711,37.18,"TEGEL, GM",FM-15,V020,"220,1,N,0067,1","01200,1,C,N","011265,1,N,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00


In [6]:
df_wetter_pkl.groupby(['Ort', 'Jahr'], as_index=True).agg(
    {     
        'Ort':['count'] #,'max','min']
    }
)

Ort
             count
Ort     Jahr      
Berlin  2007    12
        2008    12
        2009    12
        2010    12
        2011    12
        2012    12
        2013    12
        2014    12
        2015    12
        2016    12
        2017    12
        2018    12
        2019    12
Chicago 2007     4
        2008     4
        2009     4
        2010     4
        2011     4
        2012     4
        2013     4
        2014     4
        2015     4
        2016     4
        2017     4
        2018     5
        2019     4
London  2010    12
        2011    12
        2012    12
        2013    12
        2014    12
        2015    12
        2016    12
        2017    12
        2018    12
        2019    12
NewYork 2007     6
        2008     6
        2009     6
        2010     6
        2011     6
        2013     6
        2014     6
        2015     6
        2016     6
        2017    11
        2018     7
        2019     6
Tokyo   2007    20
        2008    16
        2009    16
        2010    19
        2011    15
        2012    15
        2013    16
        2014    15
        2015    20
        2016    15
        2017    15
        2018    15
        2019    14

In [7]:
df_wetter_pkl.groupby(['Ort'], as_index=True).agg(
    {     
        'Ort':['count'] #,'max','min']
    }
)

,Ort
,count
Ort,
Berlin,156
Chicago,53
London,120
NewYork,78
Tokyo,211


In [10]:
df_wetter_1 = df_wetter_pkl.drop(columns=['SOURCE','LATITUDE','LONGITUDE','REPORT_TYPE','QUALITY_CONTROL','CIG','VIS']).copy()
df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00


In [11]:
# WND/TMP/DEW/SLP Spalte beinhaltet mehrere Informationen die in einzelne Spalten getrennt werden müssen
df_wetter_1['WND_DA'] = None # WIND-OBSERVATION direction angle
df_wetter_1['WND_DA_QC'] = None # WIND-OBSERVATION direction quality code
df_wetter_1['WND_TC'] = None # WIND-OBSERVATION type code
df_wetter_1['WND_SR'] = None # WIND-OBSERVATION speed rate
df_wetter_1['WND_SR_QC'] = None # WIND-OBSERVATION speed quality code
df_wetter_1['TMP_AT'] = None # AIR-TEMPERATURE-OBSERVATION air temperature
df_wetter_1['TMP_AT_QC'] = None # AIR-TEMPERATURE-OBSERVATION air temperature quality code
df_wetter_1['DEW_PT'] = None # AIR-TEMPERATURE-OBSERVATION dew point temperature
df_wetter_1['DEW_PT_QC'] = None # AIR-TEMPERATURE-OBSERVATION dew point quality code
df_wetter_1['SLP_SLP'] = None # ATMOSPHERIC-PRESSURE-OBSERVATION sea level pressure
df_wetter_1['SLP_SLP_QC'] = None # ATMOSPHERIC-PRESSURE-OBSERVATION sea level pressure quality code

df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00,None,None,None,None,None,None,None,None,None,None,None
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00,None,None,None,None,None,None,None,None,None,None,None
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00,None,None,None,None,None,None,None,None,None,None,None
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00,None,None,None,None,None,None,None,None,None,None,None
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00,None,None,None,None,None,None,None,None,None,None,None


In [12]:
# Neue Spalten befüllen
for i, row in df_wetter_1.iterrows():
    wnd_splitted = row['WND'].split(",")
    df_wetter_1.loc[i,['WND_DA']] = wnd_splitted[0]
    df_wetter_1.loc[i,['WND_DA_QC']] = wnd_splitted[1]
    df_wetter_1.loc[i,['WND_TC']] = wnd_splitted[2]
    df_wetter_1.loc[i,['WND_SR']] = wnd_splitted[3]
    df_wetter_1.loc[i,['WND_SR_QC']] = wnd_splitted[4]

    tmp_splitted = row['TMP'].split(",")
    df_wetter_1.loc[i,['TMP_AT']] = tmp_splitted[0]
    df_wetter_1.loc[i,['TMP_AT_QC']] = tmp_splitted[1]

    dew_splitted = row['DEW'].split(",")
    df_wetter_1.loc[i,['DEW_PT']] = dew_splitted[0]
    df_wetter_1.loc[i,['DEW_PT_QC']] = dew_splitted[1]

    slp_splitted = row['SLP'].split(",")
    df_wetter_1.loc[i,['SLP_SLP']] = slp_splitted[0]
    df_wetter_1.loc[i,['SLP_SLP_QC']] = slp_splitted[1]

df_wetter_1.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","200,1,N,0036,1","+0130,1","+0100,1","99999,9",2007-09-30 07:20:00,200,1,N,0036,1,+0130,1,+0100,1,99999,9
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","210,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 07:50:00,210,1,N,0051,1,+0140,1,+0100,1,99999,9
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0051,1","+0140,1","+0110,1","99999,9",2007-09-30 08:20:00,220,1,N,0051,1,+0140,1,+0110,1,99999,9
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","230,1,N,0051,1","+0140,1","+0100,1","99999,9",2007-09-30 08:50:00,230,1,N,0051,1,+0140,1,+0100,1,99999,9
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM","220,1,N,0067,1","+0150,1","+0100,1","99999,9",2007-09-30 09:20:00,220,1,N,0067,1,+0150,1,+0100,1,99999,9


In [13]:
# WND_DA_QC Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
'''
df_wetter_1.groupby(['WND_DA_QC']).agg({
    'WND_DA_QC' : 'count'
})

,WND_DA_QC
WND_DA_QC,
1,474
5,80
9,64


In [14]:
# WND_TC Datenqualität prüfen
'''
A = Abridged Beaufort
B = Beaufort
C = Calm
H = 5-Minute Average Speed
N = Normal
R = 60-Minute Average Speed
Q = Squall
T = 180 Minute Average Speed
V = Variable
9 = Missing
(If a value of 9 appears with a wind speed of 0000, this indicates calm winds.)
'''
df_wetter_1.groupby(['WND_TC'],as_index=True).agg({
    'WND_TC' : 'count'    
})

,WND_TC
WND_TC,
9,2
C,10
N,444
V,162


In [15]:
df_wetter_1.loc[(df_wetter_1.WND_TC == '9')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
367,2014,NewYork,2014-11-02,2014-11-02 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0072,5","-0033,5","10113,5",2014-11-02 15:51:00,999,9,9,9999,9,+0072,5,-0033,5,10113,5
379,2016,NewYork,2016-11-06,2016-11-06 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0133,5","+0011,5","10228,5",2016-11-06 15:51:00,999,9,9,9999,9,+0133,5,+0011,5,10228,5


In [16]:
# WND_SR_QC Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
'''
df_wetter_1.groupby(['WND_SR_QC'],as_index=True).agg({
    'WND_SR_QC' : 'count'    
})

,WND_SR_QC
WND_SR_QC,
1,516
5,97
9,5


In [17]:
# WND_DA Datenqualität prüfen
'''
999 = Missing. If type code (below) = V, then 999 indicates variable wind direction
'''
df_wetter_1.loc[(df_wetter_1.WND_DA == '999')].groupby(['WND_DA','WND_DA_QC','WND_TC']).agg({
    'WND':'count'
})

WND
WND_DA WND_DA_QC WND_TC     
999    9         9         2
                 C        10
                 N         4
                 V        48

In [18]:
# WND_SR Datenqualität prüfen
'''
A general domain comprised of the numeric characters (0-9). 9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.WND_SR == '9999')].groupby(['WND_SR']).agg({
    'WND':'count'
})

,WND
WND_SR,
9999,5


In [19]:
df_wetter_1.loc[(df_wetter_1.WND_SR == '9999')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
367,2014,NewYork,2014-11-02,2014-11-02 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0072,5","-0033,5","10113,5",2014-11-02 15:51:00,999,9,9,9999,9,+0072,5,-0033,5,10113,5
379,2016,NewYork,2016-11-06,2016-11-06 13:30:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,9,9999,9","+0133,5","+0011,5","10228,5",2016-11-06 15:51:00,999,9,9,9999,9,+0133,5,+0011,5,10228,5
394,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10326,5",2018-11-04 17:51:00,999,9,N,9999,9,+0122,5,-0011,5,10326,5
395,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10320,5",2018-11-04 18:51:00,999,9,N,9999,9,+0122,5,-0011,5,10320,5
396,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0128,5","-0017,5","10318,5",2018-11-04 19:51:00,999,9,N,9999,9,+0128,5,-0017,5,10318,5


In [20]:
df_wetter_1.loc[(df_wetter_1.Ort == 'NewYork') & (df_wetter_1.Jahr == '2018')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
394,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10326,5",2018-11-04 17:51:00,999,9,N,9999,9,+0122,5,-0011,5,10326,5
395,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0122,5","-0011,5","10320,5",2018-11-04 18:51:00,999,9,N,9999,9,+0122,5,-0011,5,10320,5
396,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72505394728,42.7,"NY CITY CENTRAL PARK, NY US","999,9,N,9999,9","+0128,5","-0017,5","10318,5",2018-11-04 19:51:00,999,9,N,9999,9,+0128,5,-0017,5,10318,5
397,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","999,9,V,0015,5","+0128,5","+0000,5","10324,5",2018-11-04 17:51:00,999,9,V,0015,5,+0128,5,+0000,5,10324,5
398,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","999,9,N,0015,1","+0128,1","+0000,1","10324,1",2018-11-04 18:00:00,999,9,N,0015,1,+0128,1,+0000,1,10324,1
399,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","210,5,N,0026,5","+0128,5","-0006,5","10319,5",2018-11-04 18:51:00,210,5,N,0026,5,+0128,5,-0006,5,10319,5
400,2018,NewYork,2018-11-04,2018-11-04 17:00:00,72502014734,2.1,"NEWARK LIBERTY INTERNATIONAL AIRPORT, NJ US","070,5,N,0021,5","+0133,5","-0006,5","10318,5",2018-11-04 19:51:00,070,5,N,0021,5,+0133,5,-0006,5,10318,5


In [21]:
# TMP-Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
A = Data value flagged as suspect, but accepted as a good value
C = Temperature and dew point received from Automated Weather Observing System (AWOS) are reported in
whole degrees Celsius. Automated QC flags these values, but they are accepted as valid.
I = Data value not originally in data, but inserted by validator
M = Manual changes made to value based on information provided by NWS or FAA
P = Data value not originally flagged as suspect, but replaced by validator
R = Data value replaced with value computed by NCEI software
U = Data value replaced with edited value

A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.TMP_AT == '99999')] 


,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC


In [22]:
df_wetter_1.groupby(['TMP_AT_QC']).agg({'TMP_AT_QC':'count'})

,TMP_AT_QC
TMP_AT_QC,
1,516
5,102


In [23]:
# DEW-Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present
A = Data value flagged as suspect, but accepted as a good value
C = Temperature and dew point received from Automated Weather Observing System (AWOS) are reported in
whole degrees Celsius. Automated QC flags these values, but they are accepted as valid.
I = Data value not originally in data, but inserted by validator
M = Manual changes made to value based on information provided by NWS or FAA
P = Data value not originally flagged as suspect, but replaced by validator
R = Data value replaced with value computed by NCEI software
U = Data value replaced with edited value

A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.DEW == '9999')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,WND,TMP,DEW,SLP,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC


In [24]:
df_wetter_1.groupby(['DEW_PT_QC']).agg({'DEW_PT_QC':'count'})

,DEW_PT_QC
DEW_PT_QC,
1,516
5,102


In [25]:
# SLP-Datenqualität prüfen
'''
0 = Passed gross limits check
1 = Passed all quality control checks
2 = Suspect
3 = Erroneous
4 = Passed gross limits check, data originate from an NCEI data source
5 = Passed all quality control checks, data originate from an NCEI data source
6 = Suspect, data originate from an NCEI data source
7 = Erroneous, data originate from an NCEI data source
9 = Passed gross limits check if element is present

A general domain comprised of the numeric characters (0-9), a plus sign (+), and a minus sign (-). +9999 = Missing.
'''
df_wetter_1.loc[(df_wetter_1.SLP_SLP == '99999')].groupby(['SLP_SLP']).agg({'SLP_SLP':'count'}) 

,SLP_SLP
SLP_SLP,
99999,443


In [26]:
df_wetter_1.groupby(['SLP_SLP_QC']).agg({'SLP_SLP_QC':'count'}) 

,SLP_SLP_QC
SLP_SLP_QC,
1,79
5,96
9,443


<p>Auf grund der Analyse müssen unterschiedliche Dataframes angelegt werden</p>
<lu>
<li>Ein Dataframe mit den Temperaturen und Taupunkten, da hier alle Datenvorhanden sind</li>
<li>Dataframes für die Windstärke und Windrichtung, da hier einige Daten fehlen</li>
<li>Dataframe für den Luftdruck, da hier auch einige Daten fehlen</li>
</lu>

In [27]:
#df_wetter_1.loc[(df_wetter_1.SLP == '99999,9')]

In [28]:
# Dataframe kopieren und nicht benötigte Spalten entfernen
df_wetter_2 = df_wetter_1.drop(columns=['WND','TMP','DEW','SLP']).copy()
df_wetter_2.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:20:00,200,1,N,0036,1,+0130,1,+0100,1,99999,9
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:50:00,210,1,N,0051,1,+0140,1,+0100,1,99999,9
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:20:00,220,1,N,0051,1,+0140,1,+0110,1,99999,9
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:50:00,230,1,N,0051,1,+0140,1,+0100,1,99999,9
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 09:20:00,220,1,N,0067,1,+0150,1,+0100,1,99999,9


In [29]:
df_wetter_2.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:20:00,200,1,N,0036,1,+0130,1,+0100,1,99999,9
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:50:00,210,1,N,0051,1,+0140,1,+0100,1,99999,9
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:20:00,220,1,N,0051,1,+0140,1,+0110,1,99999,9
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:50:00,230,1,N,0051,1,+0140,1,+0100,1,99999,9
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 09:20:00,220,1,N,0067,1,+0150,1,+0100,1,99999,9


In [30]:
# Datentypen setzen
df_wetter_2['DEW_PT'] = df_wetter_2['DEW_PT'].astype(float)
df_wetter_2['TMP_AT'] = df_wetter_2['TMP_AT'].astype(float)
df_wetter_2['WND_SR'] = df_wetter_2['WND_SR'].astype(float)
df_wetter_2['SLP_SLP'] = df_wetter_2['SLP_SLP'].astype(float)
df_wetter_2['WND_DA'] = df_wetter_2['WND_DA'].astype(int)

df_wetter_2['ELEVATION'] = df_wetter_2['ELEVATION'].astype(float)
df_wetter_2['Jahr'] = df_wetter_2['Jahr'].astype(int)

In [31]:
df_wetter_2.loc[(df_wetter_2.Ort == 'Tokyo') & (df_wetter_2.Jahr == 2008)].head(30) # Prüfung

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
427,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 00:00:00,340,1,N,62.0,1,40.0,1,-80.0,1,99999.0,9
428,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 00:30:00,330,1,N,51.0,1,50.0,1,-90.0,1,99999.0,9
429,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 01:00:00,350,1,N,62.0,1,50.0,1,-90.0,1,99999.0,9
430,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 01:30:00,320,1,N,62.0,1,60.0,1,-110.0,1,99999.0,9
431,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:00:00,10,1,N,36.0,1,50.0,1,-110.0,1,99999.0,9
432,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:30:00,30,1,N,26.0,1,60.0,1,-130.0,1,99999.0,9
433,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:42:00,360,1,N,77.0,1,70.0,1,-100.0,1,99999.0,9
434,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 03:00:00,340,1,N,62.0,1,60.0,1,-120.0,1,99999.0,9
435,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47662099999,36.00,"TOKYO, JA",2008-02-17 00:00:00,320,1,N,31.0,1,36.0,1,-94.0,1,10222.0,1
436,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47662099999,36.00,"TOKYO, JA",2008-02-17 01:00:00,330,1,N,26.0,1,48.0,1,-126.0,1,10221.0,1


In [32]:
# Skalierung gemäß der Dokumentation anpassen:
df_wetter_2['WND_SR'] = df_wetter_2['WND_SR'] / 10
df_wetter_2['TMP_AT'] = df_wetter_2['TMP_AT'] / 10
df_wetter_2['DEW_PT'] = df_wetter_2['DEW_PT'] / 10
df_wetter_2['SLP_SLP'] = df_wetter_2['SLP_SLP'] / 10

In [33]:
df_wetter_2.loc[(df_wetter_2.Ort == 'Tokyo') & (df_wetter_2.Jahr == 2008)].head(30) # Prüfung

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
427,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 00:00:00,340,1,N,6.2,1,4.0,1,-8.0,1,9999.9,9
428,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 00:30:00,330,1,N,5.1,1,5.0,1,-9.0,1,9999.9,9
429,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 01:00:00,350,1,N,6.2,1,5.0,1,-9.0,1,9999.9,9
430,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 01:30:00,320,1,N,6.2,1,6.0,1,-11.0,1,9999.9,9
431,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:00:00,10,1,N,3.6,1,5.0,1,-11.0,1,9999.9,9
432,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:30:00,30,1,N,2.6,1,6.0,1,-13.0,1,9999.9,9
433,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 02:42:00,360,1,N,7.7,1,7.0,1,-10.0,1,9999.9,9
434,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47671099999,10.66,"TOKYO INTERNATIONAL, JA",2008-02-17 03:00:00,340,1,N,6.2,1,6.0,1,-12.0,1,9999.9,9
435,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47662099999,36.00,"TOKYO, JA",2008-02-17 00:00:00,320,1,N,3.1,1,3.6,1,-9.4,1,1022.2,1
436,2008,Tokyo,2008-02-17,2008-02-17 00:10:00,47662099999,36.00,"TOKYO, JA",2008-02-17 01:00:00,330,1,N,2.6,1,4.8,1,-12.6,1,1022.1,1


In [34]:
df_wetter_3 = df_wetter_2.copy()
df_wetter_3.head()

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
0,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:20:00,200,1,N,3.6,1,13.0,1,10.0,1,9999.9,9
1,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 07:50:00,210,1,N,5.1,1,14.0,1,10.0,1,9999.9,9
2,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:20:00,220,1,N,5.1,1,14.0,1,11.0,1,9999.9,9
3,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 08:50:00,230,1,N,5.1,1,14.0,1,10.0,1,9999.9,9
4,2007,Berlin,2007-09-30,2007-09-30 07:15:00,10382099999,37.18,"TEGEL, GM",2007-09-30 09:20:00,220,1,N,6.7,1,15.0,1,10.0,1,9999.9,9


In [35]:
# Dataframe serialisieren (Pickle) 
pickle.dump(df_wetter_3, open(data_path_weather + 'daten_wetter_step_2.p', "wb"))

In [37]:
df_wetter_3.loc[(df_wetter_3.Ort == 'Chicago')]

,Jahr,Ort,Datum,Datum_Startzeit_UTC,STATION,ELEVATION,NAME,DATE_TIME,WND_DA,WND_DA_QC,WND_TC,WND_SR,WND_SR_QC,TMP_AT,TMP_AT_QC,DEW_PT,DEW_PT_QC,SLP_SLP,SLP_SLP_QC
156,2007,Chicago,2007-10-07,2007-10-07 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2007-10-07 12:51:00,200,5,N,2.1,5,22.2,5,18.3,5,1018.3,5
157,2007,Chicago,2007-10-07,2007-10-07 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2007-10-07 13:51:00,200,5,N,2.1,5,23.9,5,18.9,5,1018.6,5
158,2007,Chicago,2007-10-07,2007-10-07 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2007-10-07 14:51:00,200,5,N,3.1,5,26.1,5,19.4,5,1018.5,5
159,2007,Chicago,2007-10-07,2007-10-07 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2007-10-07 15:00:00,200,1,N,3.1,1,26.1,1,19.4,1,1018.5,1
160,2008,Chicago,2008-10-12,2008-10-12 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2008-10-12 12:51:00,160,5,N,2.1,5,17.8,5,13.3,5,1026.4,5
161,2008,Chicago,2008-10-12,2008-10-12 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2008-10-12 13:51:00,170,5,N,3.1,5,18.9,5,12.8,5,1026.8,5
162,2008,Chicago,2008-10-12,2008-10-12 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2008-10-12 14:51:00,170,5,N,3.6,5,21.1,5,12.8,5,1026.8,5
163,2008,Chicago,2008-10-12,2008-10-12 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2008-10-12 15:00:00,170,1,N,3.6,1,21.1,1,12.8,1,1026.8,1
164,2009,Chicago,2009-10-11,2009-10-11 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2009-10-11 12:51:00,310,5,N,3.1,5,0.0,5,-5.6,5,1030.1,5
165,2009,Chicago,2009-10-11,2009-10-11 12:30:00,72534014819,186.5,"CHICAGO MIDWAY AIRPORT, IL US",2009-10-11 13:51:00,320,5,N,3.6,5,1.1,5,-5.0,5,1030.5,5
